#Programação Matemática - Atividade 2

# Modelo de Minimização de Custos para Produção de Camisetas

## Variáveis de Decisão
- \( $P_t$ \): quantidade de camisetas produzidas na semana \( $t$ \) (onde \( $t$ = 1, 2, 3, 4 \))
- \( $E_t$ \): quantidade de camisetas produzidas com horas extras na semana \( $t$ \) (onde \( $t$ = 1, 2 \))
- \( $I_t$ \): quantidade de camisetas em estoque no final da semana \( t \) (onde \( $t$ = 1, 2, 3, 4 \))

## Parâmetros
- Demanda semanal \( $D_t$ \):
  - \( $D_1$ = 5000 \)
  - \( $D_2$ = 10000 \)
  - \( $D_3$ = 30000 \)
  - \( $D_4$ = 60000 \)

- Custos de produção:
  - Semanas 1 e 2: R\$ 2,00
  - Semanas 3 e 4: R\$ 2,50
  - Custo com horas extras: R$ 2,80 (apenas nas semanas 1 e 2)

- Custo de estocagem: R$ 0,20 por unidade por semana.

- Capacidade de produção:
  - Sem horas extras: 25.000 camisetas por semana.
  - Com horas extras: 35.000 camisetas por semana (25.000 + 10.000).

## Função Objetivo
Minimizar o custo total:
\begin{equation}
Z = \sum_{t=1}^{4} C_t \cdot P_t + \sum_{t=1}^{2} (2.80 \cdot E_t) + \sum_{t=1}^{4} (0.20 \cdot I_t)
\end{equation}

onde:
- \( $C_t$ \) é o custo de produção na semana \( $t$ \):
  - \( $C_1$ = 2.00 \)
  - \( $C_2$ = 2.00 \)
  - \( $C_3$ = 2.50 \)
  - \( $C_4$ = 2.50 \)

## Restrições

1. **Capacidade de produção**:
   $P_t + E_t \leq 35000 \quad \text{para } t = 1, 2
   P_t \leq 25000 \quad \text{para } t = 3, 4$

2. **Demandas**:
   $P_t + I_{t-1} - I_t = D_t \quad \text{para } t = 1, 2, 3, 4$
   (com $I_0$ = 0 )

3. **Estoque não negativo**:
   $I_t \geq 0 \quad \text{para } t = 1, 2, 3, 4$

4. **Produção não negativa**:
   $P_t \geq 0 \quad \text{para } t = 1, 2, 3, 4$
   $E_t \geq 0 \quad \text{para } t = 1, 2$

## Modelo Completo
Minimizar $Z$:
$Z = \sum_{t=1}^{4} C_t \cdot P_t + \sum_{t=1}^{2} (2.80 \cdot E_t) + \sum_{t=1}^{4} (0.20 \cdot I_t)$

Sujeito a:

$P_t + E_t \leq 35000 \quad \text{para } t = 1, 2$

$P_t \leq 25000 \quad \text{para } t = 3, 4$

$P_t + I_{t-1} - I_t = D_t \quad \text{para } t = 1, 2, 3, 4$

$I_t \geq 0 \quad \text{para } t = 1, 2, 3, 4$

$P_t \geq 0 \quad \text{para } t = 1, 2, 3, 4$

$E_t \geq 0 \quad \text{para } t = 1, 2$


In [44]:
!pip install pulp
import pulp

In [45]:
# Definição de parametros:
periodos = [1, 2, 3, 4]
demanda = {1: 5000, 2: 10000, 3: 30000, 4: 60000}
custosProducao = {1: 2.00, 2: 2.00, 3: 2.50, 4: 2.50}
capacidadeProducao = 25000
capacidadeProducaoHorasExtras = 10000

In [46]:
modelo = pulp.LpProblem("ProducaoCamisetas", pulp.LpMinimize)

In [47]:
qtdProduzida = pulp.LpVariable.dicts("qtdProduzida", periodos, lowBound=0, cat='Integer')
qtdHorasExtras = pulp.LpVariable.dicts("qtdHorasExtras", [1, 2], lowBound=0, cat='Integer')
qtdEmEstoque = pulp.LpVariable.dicts("qtdEmEstoque", periodos, lowBound=0, cat='Integer')

In [48]:
modelo += pulp.lpSum(custosProducao[t] * qtdProduzida[t] for t in periodos) + pulp.lpSum(2.80 * qtdHorasExtras[t] for t in [1, 2]) + pulp.lpSum(0.2 * qtdEmEstoque[t] for t in periodos)

## Restrições

In [49]:
# Restrições
for t in periodos:
    modelo += qtdProduzida[t] <= capacidadeProducao

    if t in [1, 2]:
        modelo += qtdHorasExtras[t] <= capacidadeProducaoHorasExtras

    if t == 1:
        modelo += qtdProduzida[t] + qtdHorasExtras[t] - qtdEmEstoque[t] == demanda[t]
    else:
        if t == 2:
            modelo += qtdProduzida[t] + qtdHorasExtras[t] + qtdEmEstoque[t-1] - qtdEmEstoque[t] == demanda[t]
        else:
            modelo += qtdProduzida[t] + qtdEmEstoque[t-1] - qtdEmEstoque[t] == demanda[t]

    modelo += qtdEmEstoque[t] >= 0
    modelo += qtdProduzida[t] >= 0
    if t in [1, 2]:
        modelo += qtdHorasExtras[t] >= 0

In [50]:
status = modelo.solve()

In [51]:
print(f"Status: {pulp.LpStatus[status]}")
for i in periodos:
    print(f"Semana {i}: ",end=' ')
    print(f" Camisetas produzidas: {qtdProduzida[i].value()}", end='  ')
    if i in [1, 2]:
        print(f"Quantidade de horas extras: {qtdHorasExtras[i].value()}", end=' ')
    print(f"Estoque:  {qtdEmEstoque[i].value()}")

print(f"Custo: {modelo.objective.value()}")

Status: Optimal
Semana 1:   Camisetas produzidas: 25000.0  Quantidade de horas extras: 0.0 Estoque:  20000.0
Semana 2:   Camisetas produzidas: 25000.0  Quantidade de horas extras: 5000.0 Estoque:  40000.0
Semana 3:   Camisetas produzidas: 25000.0  Estoque:  35000.0
Semana 4:   Camisetas produzidas: 25000.0  Estoque:  0.0
Custo: 258000.0
